In [1]:
from neo4j import GraphDatabase
from CurriculumDB.Modelsn4j import *
import docx
import os
import openpyxl

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "bolt://localhost:7687"
AUTH = ("curriculum", "mycurriculum")

driver = GraphDatabase.driver(URI, auth=AUTH)
print(driver)
print(driver.verify_connectivity())
#help(driver.verify_connectivity)

factoryname='curriculumdb'

factory = CurriculumFactory(driver, factoryname)

None


In [2]:
# extract all programme ILOs
# Extract all modules and module ILOs
#Extract all RSB benchmarks
# Extract NIBLSE

pilos = factory.get_all_elements('ProgrammeILO')
modules = factory.get_all_elements('Module')
rsb = factory.get_all_elements('RSBCriterion')
niblse = factory.get_all_elements('NIBLSEcompetency')

In [ ]:
niblse[0].params

In [ ]:
from openpyxl.styles import Font


headerfont = Font(color="000088",bold=True, name='Arial', size=14)
wrapit = openpyxl.styles.Alignment(wrap_text=True)
titlefont= Font(color='000000',name="Arial", bold=True, size=16)
wb = openpyxl.Workbook()
wb.create_sheet('RSBCriteria')
sheet=wb['RSBCriteria']
sheet.append(['ID', 'code', 'criterion','section', 'subsection', 
              'criterion_text', 'section_text', 'subsection_text', 'notes'])
for r in rsb:
        code = f"{r.params.get('criterion',' ')} {r.params.get('section',' ')} {r.params.get('subsection',' ')}: {r.params.get('criterion_text',' ')}: {r.params.get('section_text',' ')}: {r.params.get('subsection_text',' ')}"
        sheet.append([r.element_id, code,r.params.get('criterion',' '),r.params.get('section',' '),
                      r.params.get('subsection',' '),r.params.get('criterion_text',' '),r.params.get('section_text',' '),
                      r.params.get('subsection_text',' '),r.params.get('notes',' ') ])
rdv = openpyxl.worksheet.datavalidation.DataValidation(type="list", formula1=f"RSBCriteria!$B$2:$B${sheet.max_row}" , allow_blank=True)
rdv.prompt="Map to RSB Criteria"
wb.create_sheet('ProgrammeILOs')
wb.remove(wb['Sheet'])
sheet=wb['ProgrammeILOs']
sheet.append(['ILO id','Programme ILO', 'RSB Criterion'])
sheet["A1"].font=headerfont
sheet["B1"].font=headerfont
sheet["C1"].font=headerfont
sheet.add_data_validation(rdv)
for p in pilos:
    sheet.append([p.element_id, p.params['outcome']])
    rdv.add(sheet[f'C{sheet.max_row}'])
    sheet[f'B{sheet.max_row}'].alignment=wrapit
    sheet[f'C{sheet.max_row}'].alignment=wrapit
sheet.column_dimensions['B'].width=120
sheet.column_dimensions['C'].width=80
pcount = sheet.max_row-1
print(pcount)
pdv = openpyxl.worksheet.datavalidation.DataValidation(type="list", formula1=f"ProgrammeILOs!$B$2:$B${pcount+1}" , allow_blank=True)
pdv.prompt="Map to Programme ILO"

wb.create_sheet('NIBLSE')
sheet=wb['NIBLSE']
sheet.append(['ID', 'code', 'competency', 'descriptor', 'explanation', 'examples'])
for i in "ABCDEF":
    sheet[f"{i}1"].font=headerfont
for n in niblse:
    code = f"{n.params.get('competency',' ')}, {n.params.get('descriptor',' ')}"
    sheet.append([n.element_id, code,n.params.get('competency',' ') , n.params.get('descriptor',' '),
                  n.params.get('explanation',' '), n.params.get('examples',' ')])
ndv = openpyxl.worksheet.datavalidation.DataValidation(type="list", formula1=f"NIBLSE!$B$2:$B${sheet.max_row}", allow_blank=True)
ndv.prompt="Map to NIBLSE competency"
for m in modules:
    wb.create_sheet(m.params['code'].upper())
    
    sheet = wb[m.params['code'].upper()]
    sheet.add_data_validation(pdv)
    sheet.add_data_validation(ndv)
    sheet.append([m.params.get('code',''),m.params.get('name','')])
    sheet.append([])
    sheet.append(['ID', 'ILO', 'Programme ILO', 'NIBLSE'])
    for i in "ABCD":
        sheet[f"{i}3"].font=headerfont
    sheet.column_dimensions['B'].width=80
    sheet.column_dimensions['C'].width=80
    sheet.column_dimensions['D'].width=80
    
    
    for i in m.ILO:
        sheet.append([i, m.ILO[i][0].get('outcome')])
        icell=sheet[f"B{sheet.max_row}"]
        icell.alignment=wrapit
        pcell = sheet[f"C{sheet.max_row}"]
        pcell.alignment=wrapit
        ncell = sheet[f"D{sheet.max_row}"]
        ncell.alignment=wrapit
        pdv.add(pcell)
        ndv.add(ncell)
wb.save('modulemap.xlsx')

In [ ]:
help(openpyxl.worksheet.datavalidation.DataValidation)